In [1]:
import json
lines = ''
with open("data.json", encoding="utf-8") as f:
    lines = ''.join(f.readlines())

allData = json.loads(lines)


In [2]:
# for game in allData:
#     print(game['itadPlain'])

In [3]:
# Sample of what the input data looks like
allData[0]

{'steamId': '524220',
 'itadPlain': 'nierautomata',
 'itad.position': 1,
 'itad.rank': 1,
 'itad.sales': [[1456005749000, 19.99],
  [1458665932000, 19.99],
  [1458666774000, 11.99],
  [1459270405000, 11.99],
  [1459271668000, 19.99],
  [1465572067000, 19.99],
  [1465923310000, 19.99],
  [1465924595000, 11.99],
  [1466182562000, 11.99],
  [1466183182000, 19.99],
  [1466682484000, 19.99],
  [1466683768000, 19.99],
  [1466700744000, 19.99],
  [1466710282000, 11.99],
  [1466752198000, 11.99],
  [1466756890000, 11.99],
  [1467055249000, 11.99],
  [1467061585000, 11.99],
  [1467115991000, 11.99],
  [1467649507000, 11.99],
  [1467654345000, 19.99],
  [1469789837000, 19.99],
  [1469789838000, 15.78],
  [1470084012000, 15.78],
  [1470085211000, 19.99],
  [1470110417000, 19.99],
  [1471547740000, 19.99],
  [1471551336000, 10.99],
  [1471886137000, 10.99],
  [1471887930000, 19.99],
  [1477680344000, 19.99],
  [1477682138000, 9.99],
  [1478034971000, 9.99],
  [1478036747000, 19.99],
  [14782537160

In [4]:
allCategoriesSet = set()
allGenresSet = set()

for game in allData:
    allCategoriesSet.update(game['steam.categories_description'])
    allGenresSet.update(game['steam.genres_description'])

allCategories = sorted(allCategoriesSet)
allGenres = sorted(allGenresSet)

In [5]:
from copy import copy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime, timezone, timedelta, date

analyzer = SentimentIntensityAnalyzer()
# We consider all types of spanish (spain, brazil) the same. This is because there are many ways to phrase it, and it is hard to clean.

SUPPORTED_LANGUAGES = ['english', 'french', 'german', 'dutch', 'czech', 'hungarian', 'italian', 'japanese', 'polish', 'bulgarian', 'danish', 'spanish', 'finnish', 'greek', 'korean', 'norwegian', 'portuguese', 'romanian', 'russian', 'simplified chinese', 'traditional chinese', 'swedish', 'thai', 'turkish', 'ukrainian', 'vietnamese', 'arabic']

def cleanup_data(game):
    """
    Takes a single game object, and 1-hot encodes:
      - Supported Languages
      - Genres description
      - Categories description
    and runs sentiment analysis on:
      - short description
      - detailed description
      - about the game
    :param game:
    :return:
    """
    # Clone the data so we can run this multiple times on the same data
    flattened_game = copy(game)

    # Languages
    lang_str = flattened_game["steam.supported_languages"]
    for language in SUPPORTED_LANGUAGES:
        attrib_name = f"steam.supported_languages.{language.replace(' ', '_')}".lower()
        if language in lang_str:
            flattened_game[attrib_name] = 1
        else:
            flattened_game[attrib_name] = 0
    flattened_game.pop('steam.supported_languages', None)

    # Genres
    for genre in allGenres:
        attrib_name = f"steam.genres.{genre.replace(' ', '_')}".lower()
        if genre in flattened_game['steam.genres_description']:
            flattened_game[attrib_name] = 1
        else:
            flattened_game[attrib_name] = 0
    flattened_game.pop('steam.genres_description', None)

    # Categories
    # 'steam.categories_description'
    for category in allCategories:
        attrib_name = f"steam.categories.{category.replace(' ', '_')}".lower()
        if category in flattened_game['steam.categories_description']:
            flattened_game[attrib_name] = 1
        else:
            flattened_game[attrib_name] = 0
    flattened_game.pop('steam.categories_description', None)

    # Too many developers are unique. Let's try to simplify it a bit and just count the number of devs.
    flattened_game['steam.developers'] = len(flattened_game['steam.developers'])


    # Replace the 'steam.detailed_description', 'steam.about_the_game', 'steam.short_description' with their sentiment analysis
    # {'neg': 0.11, 'neu': 0.803, 'pos': 0.087, 'compound': -0.8976}
    LONG_FORM_TEXT_ATTRIBS = ['steam.detailed_description', 'steam.about_the_game', 'steam.short_description']
    SENTIMENT_OPTIONS = ['neg', 'neu', 'pos', 'compound']
    for attrib in LONG_FORM_TEXT_ATTRIBS:
        analysis = analyzer.polarity_scores(flattened_game[attrib])
        for sentiment in SENTIMENT_OPTIONS:
            flattened_game[f"{attrib}.{sentiment}"] = analysis[sentiment]
        flattened_game.pop(attrib, None)


    return flattened_game

def get_steam_sale_date_set():
    # We only consider steam sales from https://steamdb.info/sales/history/ that are likely to affect many games
    # These are things like the annual lunar, spring, summer, autumn, halloween, winter sales if they are present (varies slightly by year).
    # This means we leave out end of year awards sales, and niche genre sales (e.g. VR, Polish independence, Golden week) because those sales do not repeat or only affect a small subset of games.
    STEAM_SALE_DATES = [
        # [Y,M,D,Duration(days)]
        [2016,6,23,11],
        [2016,10,28,4],
        [2016,11,23,6],
        [2016,12,22,11],
        [2017,6,22,13],
        [2017,10,26,6],
        [2017,11,22,6],
        [2017,12,21,14],
        [2018,2,15,4],
        [2018,5,24,4],
        [2018,6,21,14],
        [2018,10,29,3],
        [2018,11,21,6],
        [2018,12,20,14],
        [2019,2,4,7],
        [2019,5,24,4],
        [2019,6,25,14],
        [2019,10,28,4],
        [2019,11,26,7],
        [2019,12,19,14],
        [2020,1,23,4],
        [2020,5,21,7],
        [2020,6,25,14],
        [2020,10,29,4],
        [2020,11,25,6],
        [2020,12,22,14],
        [2021,2,11,4],
        [2021,6,24,14],
        [2021,10,28,4],
        [2021,12,22,14],
        [2022,1,27,7],
    ]
    # Use a set for constant r/w speed
    saleDates = set()

    for sale in STEAM_SALE_DATES:
        [y, m, d, duration] = sale
        saleDate = date(y, m, d)
        for i in range(0, duration):
            saleDates.add(saleDate)
            saleDate += timedelta(days=1)

    return saleDates

saleDates = get_steam_sale_date_set()


# Pass in allData here
def find_day_range_and_preprocess(data):
    """
    I want it to be able to detect Data that is:
        - seasonal (steam sales)
        - price decreases after a long time (would this be a trend?)
        - absolute time
        - based off of the game's attributes
    This means, my data should probably have:
        - days since launch column
        - isSteamSale (dates from https://steamdb.info/sales/history/)
        - Year
        - Month
        - Day
        - Add column for % of launch price. This might be more effective at predicting instead of raw number.

    :param data:
    :return:
    """

    daysSet = set()
    timesSet = set()
    timestamps = set()
    for game_obj in data:
        sales = game_obj['itad.sales']
        timestamps.update([row[0] for row in sales])

    for timestamp in timestamps:
        dt_val = datetime.fromtimestamp(int(timestamp) / 1000,tz=timezone.utc)
        timesSet.add(dt_val.time())
        daysSet.add(dt_val.date())

    return

def forward_fill_sales(game):
    sales = game['itad.sales']
    eachDayRecord = []
    [releaseTimestamp, releasePrice] = sales[0]

    # Will never be affected by daylight savings or leap things.
    releaseDt = datetime.fromtimestamp(int(releaseTimestamp) / 1000,tz=timezone.utc).replace(hour=0, minute=30, second=0, microsecond=0)
    workingDay = releaseDt
    # Fill out stuff until the last day's price change
    for sale in sales:
        [saleTimestamp, salePrice] = sale
        saleTimestamp /= 1000
        # some data is bad
        if salePrice == 'NA':
            continue
        while workingDay.timestamp() <= saleTimestamp:
            eachDayRecord.append({
                'timestamp': workingDay.timestamp(),
                'year': workingDay.year,
                'month': workingDay.month,
                'day': workingDay.day,
                'daysSinceLaunch': (workingDay - releaseDt).days,
                'price': salePrice,
                'percentOfReleasePrice': salePrice / releasePrice,
                'isSteamSale':  workingDay.date() in saleDates,
            })

            # move 1 day into the future, then set the time to 12:30 AM
            workingDay += timedelta(days=1)
            workingDay = workingDay.replace(hour=0, minute=30, second=0, microsecond=0)
    # Due to the way the data was scraped, there is no need to forward fill.
    return eachDayRecord


cleanData = []
numGames = len(allData)
for ind, game in enumerate(allData):
    if ind % 100 == 0:
        print(f"Game {ind}/{numGames}")
    modified_game = cleanup_data(game)
    modified_game['itad.sales'] = forward_fill_sales(modified_game)
    cleanData.append(modified_game)




Game 0/2227
Game 100/2227
Game 200/2227
Game 300/2227
Game 400/2227
Game 500/2227
Game 600/2227
Game 700/2227
Game 800/2227
Game 900/2227
Game 1000/2227
Game 1100/2227
Game 1200/2227
Game 1300/2227
Game 1400/2227
Game 1500/2227
Game 1600/2227
Game 1700/2227
Game 1800/2227
Game 1900/2227
Game 2000/2227
Game 2100/2227
Game 2200/2227


In [6]:


# We use an off the shelf sentiment analysis since this may play a part in predicting the price, but I kinda doubt it.
# Putting this here in case it actually does.

# with open('cleanDataReadable.json', 'w', encoding='utf-8') as f:
#     json.dump(cleanData, f, ensure_ascii=False, indent=4)
#
# with open('cleanData.json', 'w', encoding='utf-8') as f:
#     json.dump(cleanData, f, ensure_ascii=False)

In [7]:
# Everything has standardized keys. let's list them all.
keys = cleanData[0].keys()
keys

dict_keys(['steamId', 'itadPlain', 'itad.position', 'itad.rank', 'itad.sales', 'steam.name', 'steam.steam_appid', 'steam.required_age', 'steam.is_free', 'steam.dlc', 'steam.reviews', 'steam.pc_requirements', 'steam.mac_requirements', 'steam.linux_requirements', 'steam.developers', 'steam.controller_support', 'steam.metacritic.score', 'steam.drm_consolidated_CrackProof', 'steam.drm_consolidated_Denuvo', 'steam.drm_consolidated_EA', 'steam.drm_consolidated_Games for Windows Live', 'steam.drm_consolidated_GameShield', 'steam.drm_consolidated_Kalypso', 'steam.drm_consolidated_SecuROM', 'steam.drm_consolidated_Ubisoft', 'steam.drm_consolidated_NA', 'steam.supported_languages.english', 'steam.supported_languages.french', 'steam.supported_languages.german', 'steam.supported_languages.dutch', 'steam.supported_languages.czech', 'steam.supported_languages.hungarian', 'steam.supported_languages.italian', 'steam.supported_languages.japanese', 'steam.supported_languages.polish', 'steam.supported_la

In [8]:
import csv

# These will be our columns.
with open('data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = list(cleanData[0].keys())
    fieldnames.remove('itad.sales')
    fieldnames.extend(['steam.ext_user_account_notice', 'timestamp', 'year', 'month', 'day', 'daysSinceLaunch', 'price', 'percentOfReleasePrice', 'isSteamSale'])

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    dataLen = len(cleanData)
    for ind, data in enumerate(cleanData):
        if ind % 50 == 0:
            print(f"Game {ind}/{dataLen}")
        try:
            for day in data['itad.sales']:
                modified_data = copy(data)
                modified_data.pop('itad.sales', None)
                writer.writerow({'steam.ext_user_account_notice': False, 'steam.metacritic.score': -1, **modified_data, **day})
        except:
            print("Error on: ", data)

Game 0/2227
Game 50/2227
Game 100/2227
Game 150/2227
Game 200/2227
Game 250/2227
Game 300/2227
Game 350/2227
Game 400/2227
Game 450/2227
Game 500/2227
Game 550/2227
Game 600/2227
Game 650/2227
Game 700/2227
Game 750/2227
Game 800/2227
Game 850/2227
Game 900/2227
Game 950/2227
Game 1000/2227
Game 1050/2227
Game 1100/2227
Game 1150/2227
Game 1200/2227
Game 1250/2227
Game 1300/2227
Game 1350/2227
Game 1400/2227
Game 1450/2227
Game 1500/2227
Game 1550/2227
Game 1600/2227
Game 1650/2227
Game 1700/2227
Game 1750/2227
Game 1800/2227
Game 1850/2227
Game 1900/2227
Game 1950/2227
Game 2000/2227
Game 2050/2227
Game 2100/2227
Game 2150/2227
Game 2200/2227
